For at køre de følgende notebooks, hvis du ikke allerede har gjort det, skal du implementere en model, der bruger `text-embedding-ada-002` som basismodel og sætte implementeringsnavnet inde i .env-filen som `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Næste trin er at indlæse Embedding-indekset i en Pandas DataFrame. Embedding-indekset er gemt i en JSON-fil kaldet `embedding_index_3m.json`. Embedding-indekset indeholder embedding for hver af YouTube-transskriptionerne frem til slutningen af oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Dernæst skal vi oprette en funktion kaldet `get_videos`, som vil søge i Embedding-indekset efter forespørgslen. Funktionen returnerer de 5 bedste videoer, der er mest lignende forespørgslen. Funktionen fungerer som følger:

1. Først oprettes en kopi af Embedding-indekset.
2. Dernæst beregnes Embedding for forespørgslen ved hjælp af OpenAI Embedding API.
3. Derefter oprettes en ny kolonne i Embedding-indekset kaldet `similarity`. Kolonnen `similarity` indeholder cosinus-ligheden mellem forespørgsels-Embedding og Embedding for hvert videosegment.
4. Dernæst filtreres Embedding-indekset efter kolonnen `similarity`. Embedding-indekset filtreres, så det kun inkluderer videoer, der har en cosinus-lighed større end eller lig med 0,75.
5. Endelig sorteres Embedding-indekset efter kolonnen `similarity`, og de 5 bedste videoer returneres.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Denne funktion er meget simpel, den udskriver bare resultaterne af søgeforespørgslen.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Først indlæses Embedding-indekset i en Pandas Dataframe.  
2. Derefter bliver brugeren bedt om at indtaste en forespørgsel.  
3. Så kaldes funktionen `get_videos` for at søge i Embedding-indekset efter forespørgslen.  
4. Til sidst kaldes funktionen `display_results` for at vise resultaterne til brugeren.  
5. Brugeren bliver derefter bedt om at indtaste en ny forespørgsel. Denne proces fortsætter, indtil brugeren indtaster `exit`.  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.da.png)  

Du vil blive bedt om at indtaste en forespørgsel. Indtast en forespørgsel og tryk på enter. Applikationen vil returnere en liste over videoer, der er relevante for forespørgslen. Applikationen vil også returnere et link til det sted i videoen, hvor svaret på spørgsmålet findes.  

Her er nogle forespørgsler, du kan prøve:  

- Hvad er Azure Machine Learning?  
- Hvordan fungerer konvolutionsneurale netværk?  
- Hvad er et neuralt netværk?  
- Kan jeg bruge Jupyter Notebooks med Azure Machine Learning?  
- Hvad er ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, bedes du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det oprindelige dokument på dets modersmål bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
